<a href="https://colab.research.google.com/github/yasis36/Wheat-leaf-disease-classification-using-CNN-/blob/main/Wheat_leaf_disease_classification_using_Transfer_Learning_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from tensorflow.keras.applications import VGG19, EfficientNetB0, ResNet50, DenseNet121
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import plot_model
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
import seaborn as sns
import os

# Define data paths
train_data_dir =  "/content/drive/MyDrive/Final Dataset/Wheat leaf Disease classification/train"
val_data_dir = "/content/drive/MyDrive/Final Dataset/Wheat leaf Disease classification/validation"

# Define image dimensions
img_width, img_height = 224, 224

# Data Augmentation (optional but recommended)
train_datagen = ImageDataGenerator(rescale=1./255,
                                  fill_mode='nearest',brightness_range=[0.8,1.5],channel_shift_range=0.7, horizontal_flip=True)
val_datagen = ImageDataGenerator(rescale=1./255,
                                )

# Load training and validation data
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=32,
    class_mode='categorical'
)
val_generator = val_datagen.flow_from_directory(
    val_data_dir,
    target_size=(img_width, img_height),
    batch_size=32,
    class_mode='categorical'
)

# Define base models (pre-trained networks)
def create_model(base_model_name):
  if base_model_name == "VGG19":
    base_model = VGG19(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))
  elif base_model_name == "EfficientNetB0":
    base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))
  elif base_model_name == "ResNet50":
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))
  elif base_model_name == "DenseNet121":
    base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))

  else:
    print("Invalid base model name. Choose from VGG19, EfficientNetB0, ResNet50, DenseNet121")
    return None
  # Freeze base model layers
  for layer in base_model.layers:
    layer.trainable = False

  # Add custom layers
  x = base_model.output
  x = Flatten()(x)
  x = Dense(256, activation='relu')(x)
  x = Dropout(0.5)(x)
  predictions = Dense(4, activation='softmax')(x)

  # Create final model
  model = Model(inputs=base_model.input, outputs=predictions)
  model.summary()
  # Compile the model
  model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])
  return model
#vgg19_model = create_model("VGG19")
#historyV=vgg19_model.fit(train_generator, epochs=25, validation_data=val_generator)

#efficientnet_model = create_model("EfficientNetB0")
#historyE=efficientnet_model.fit(train_generator, epochs=25, validation_data=val_generator)

resnet_model = create_model("ResNet50")
historyR=resnet_model.fit(train_generator, epochs=5, validation_data=val_generator)

#densenet_model = create_model("DenseNet121")
#historyD=densenet_model.fit(train_generator, epochs=25, validation_data=val_generator)
# Plot training & validation accuracy values

    # Plot training and validation curves
plt.figure(figsize=(10, 6))

 # Accuracy
plt.subplot(2, 1, 1)
plt.plot(historyR.history['accuracy'], label='Training Accuracy')
plt.plot(historyR.history['val_accuracy'], label='Validation Accuracy')
plt.title('Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

    # Loss
plt.subplot(2, 1, 2)
plt.plot(historyR.history['loss'], label='Training Loss')
plt.plot(historyR.history['val_loss'], label='Validation Loss')
plt.title('Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()


# Generate predictions
val_datagen.reset()
predictions = resnet_model.predict(val_datagen)
predicted_classes = np.argmax(predictions, axis=1)

# Get true labels
true_classes = val_datagen.classes
class_labels = list(val_datagen.class_indices.keys())

# Compute confusion matrix
conf_matrix = confusion_matrix(true_classes, predicted_classes)

# Plot confusion matrix
plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=class_labels, yticklabels=class_labels)
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.show()

# Print classification report
report = classification_report(true_classes, predicted_classes, target_names=class_labels)
print(report)


Found 10560 images belonging to 4 classes.
Found 2640 images belonging to 4 classes.
Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 230, 230, 3)          0         ['input_2[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 112, 112, 64)         9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 112, 112, 64)         256       ['conv1_conv[0][0]']          
 on)   

NameError: name 'SGD' is not defined